In [7]:
import pyspark
from pyspark import SparkContext                # sparkcontext object 
sc =SparkContext('local','appname')              # local is network name and second is appname
                          #to store data we make object of rdd

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=appname, master=local) created by __init__ at <ipython-input-1-a4bf79ae9b16>:3 

In [5]:
FAT drawback, hdfs came(costly) cluster of 6 computers 
skela works as sql (for unmanaged data) 
spark is written in skela

SyntaxError: invalid syntax (<ipython-input-5-288dcdbde0a2>, line 1)

In [8]:
nums= sc.parallelize([13,2,3,4,5,6,7,8,9])                 #parallelize create object of rdd

In [11]:
nums.take(5)
print(nums.collect())                 # collect will show all the data 

[13, 2, 3, 4, 5, 6, 7, 8, 9]


In [10]:
squared = nums.map(lambda x: x*x).collect()                  # map will create loop itself
for num in squared:
    print('%i ' % (num))

169 
4 
9 
16 
25 
36 
49 
64 
81 


SQLContext
A more convenient way is to use the DataFrame. SparkContext is already set, you can use it to create the dataFrame. You also need to declare the SQLContext

SQLContext allows connecting the engine with different data sources. It is used to initiate the functionalities of Spark SQL.

In [5]:
from pyspark.sql import Row
from pyspark.sql import SQLContext
#from  pyspark import SparkContext

In [6]:
#sc.stop()
#sc=SparkContext()
sqlContext = SQLContext(sc)

In [7]:
list_p=[('John',19),('Smith',29),('Adam',35),('Henry',50)]
list_p

[('John', 19), ('Smith', 29), ('Adam', 35), ('Henry', 50)]

Build a RDD(Resilient Distributed Dataset)

In [8]:
rdd = sc.parallelize(list_p)
rdd

ParallelCollectionRDD[3] at parallelize at PythonRDD.scala:195

In [9]:
ppl=rdd.map(lambda x: Row(name=x[0], age=int(x[1])))               # 
ppl

PythonRDD[4] at RDD at PythonRDD.scala:53

In [10]:
#Create a DataFrame context
#sqlContext.createDataFrame(ppl)
#list_p = [('John',19),('Smith',29),('Adam',35),('Henry',50)]
#rdd = sc.parallelize(list_p)
#ppl = rdd.map(lambda x: Row(name=x[0], age=int(x[1])))
DF_ppl = sqlContext.createDataFrame(ppl)

In [11]:
DF_ppl.printSchema()                  # same as describe in sql

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



In [12]:
DF_ppl.show()

+---+-----+
|age| name|
+---+-----+
| 19| John|
| 29|Smith|
| 35| Adam|
| 50|Henry|
+---+-----+



In [13]:
from pyspark.sql import SQLContext
url = "adult.csv"
from pyspark import SparkFiles                    # sparkfiles is used to read large files


In [14]:
sc.addFile(url)
sqlContext = SQLContext(sc)

In [15]:
df = sqlContext.read.csv(SparkFiles.get("adult.csv"), 
                         header=True,inferSchema=True)

In [16]:
df.printSchema()    # if inferschema is false then all datatypes becomes string and if header is false then header name like age 
                               # workclass will not be displayed , their address will be display

root
 |-- age: integer (nullable = true)
 |-- workclass: string (nullable = true)
 |-- fnlwgt: integer (nullable = true)
 |-- education: string (nullable = true)
 |-- educational-num: integer (nullable = true)
 |-- marital-status: string (nullable = true)
 |-- occupation: string (nullable = true)
 |-- relationship: string (nullable = true)
 |-- race: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- capital-gain: integer (nullable = true)
 |-- capital-loss: integer (nullable = true)
 |-- hours-per-week: integer (nullable = true)
 |-- native-country: string (nullable = true)
 |-- income: string (nullable = true)



In [17]:
df.show(2)

+---+---------+------+---------+---------------+------------------+-----------------+------------+-----+------+------------+------------+--------------+--------------+------+
|age|workclass|fnlwgt|education|educational-num|    marital-status|       occupation|relationship| race|gender|capital-gain|capital-loss|hours-per-week|native-country|income|
+---+---------+------+---------+---------------+------------------+-----------------+------------+-----+------+------------+------------+--------------+--------------+------+
| 25|  Private|226802|     11th|              7|     Never-married|Machine-op-inspct|   Own-child|Black|  Male|           0|           0|            40| United-States| <=50K|
| 38|  Private| 89814|  HS-grad|              9|Married-civ-spouse|  Farming-fishing|     Husband|White|  Male|           0|           0|            50| United-States| <=50K|
+---+---------+------+---------+---------------+------------------+-----------------+------------+-----+------+------------+-

In [18]:
df_string = sqlContext.read.csv(SparkFiles.get("adult.csv"), header=True, inferSchema=  False)
df_string.printSchema()

root
 |-- age: string (nullable = true)
 |-- workclass: string (nullable = true)
 |-- fnlwgt: string (nullable = true)
 |-- education: string (nullable = true)
 |-- educational-num: string (nullable = true)
 |-- marital-status: string (nullable = true)
 |-- occupation: string (nullable = true)
 |-- relationship: string (nullable = true)
 |-- race: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- capital-gain: string (nullable = true)
 |-- capital-loss: string (nullable = true)
 |-- hours-per-week: string (nullable = true)
 |-- native-country: string (nullable = true)
 |-- income: string (nullable = true)



In [19]:
#To convert the continuous variable in the right format,
#you can use recast the columns. You can use withColumn 
#to tell Spark which column to operate the transformation.
# Import all from `sql.types`
from pyspark.sql.types import *

In [20]:
# Write a custom function to convert the data type of
#DataFrame columns
def convertColumn(df, names, newType):
    for name in names: 
        df = df.withColumn(name, df[name].cast(newType))
    return df 
# List of continuous features
CONTI_FEATURES  = ['age', 'fnlwgt','capital-gain', 'educational-num','capital-loss', 'hours-per-week']
# Convert the type
df_string = convertColumn(df_string, CONTI_FEATURES,FloatType())
# Check the dataset
df_string.printSchema()

root
 |-- age: float (nullable = true)
 |-- workclass: string (nullable = true)
 |-- fnlwgt: float (nullable = true)
 |-- education: string (nullable = true)
 |-- educational-num: float (nullable = true)
 |-- marital-status: string (nullable = true)
 |-- occupation: string (nullable = true)
 |-- relationship: string (nullable = true)
 |-- race: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- capital-gain: float (nullable = true)
 |-- capital-loss: float (nullable = true)
 |-- hours-per-week: float (nullable = true)
 |-- native-country: string (nullable = true)
 |-- income: string (nullable = true)



In [21]:
from pyspark.ml.feature import StringIndexer
#stringIndexer = StringIndexer(inputCol="label", outputCol="newlabel")
#model = stringIndexer.fit(df)
#df = model.transform(df)
df.printSchema()

root
 |-- age: integer (nullable = true)
 |-- workclass: string (nullable = true)
 |-- fnlwgt: integer (nullable = true)
 |-- education: string (nullable = true)
 |-- educational-num: integer (nullable = true)
 |-- marital-status: string (nullable = true)
 |-- occupation: string (nullable = true)
 |-- relationship: string (nullable = true)
 |-- race: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- capital-gain: integer (nullable = true)
 |-- capital-loss: integer (nullable = true)
 |-- hours-per-week: integer (nullable = true)
 |-- native-country: string (nullable = true)
 |-- income: string (nullable = true)



Describe the data
To get a summary statistics, of the data, you can use describe(). It will compute the :

count
mean
standarddeviation
min
max

In [22]:
df.select('age','fnlwgt','education').show(5)	         # selected coloumns will be shown
#df.select('*').show()                                  # * show all fields

+---+------+------------+
|age|fnlwgt|   education|
+---+------+------------+
| 25|226802|        11th|
| 38| 89814|     HS-grad|
| 28|336951|  Assoc-acdm|
| 44|160323|Some-college|
| 18|103497|Some-college|
+---+------+------------+
only showing top 5 rows



In [23]:
df.groupBy("education").count().sort("count",ascending=False).show()           # just like in sql , we can group here 

+------------+-----+
|   education|count|
+------------+-----+
|     HS-grad|15784|
|Some-college|10878|
|   Bachelors| 8025|
|     Masters| 2657|
|   Assoc-voc| 2061|
|        11th| 1812|
|  Assoc-acdm| 1601|
|        10th| 1389|
|     7th-8th|  955|
| Prof-school|  834|
|         9th|  756|
|        12th|  657|
|   Doctorate|  594|
|     5th-6th|  509|
|     1st-4th|  247|
|   Preschool|   83|
+------------+-----+



In [24]:
 df.describe()                # describe will give summary

DataFrame[summary: string, age: string, workclass: string, fnlwgt: string, education: string, educational-num: string, marital-status: string, occupation: string, relationship: string, race: string, gender: string, capital-gain: string, capital-loss: string, hours-per-week: string, native-country: string, income: string]

In [25]:
df.describe('capital-gain').show()

+-------+------------------+
|summary|      capital-gain|
+-------+------------------+
|  count|             48842|
|   mean|1079.0676262233324|
| stddev| 7452.019057655406|
|    min|                 0|
|    max|             99999|
+-------+------------------+



In [26]:
#df.select('label').show()
#df.crosstab('age', 'educational').sort("age-label").show()

Drop column
There are two intuitive API to drop columns:

drop(): Drop a column
dropna(): Drop NA's
Below you drop the column education_num

In [1]:
df.drop('educational-num').columns

NameError: name 'df' is not defined

Filter data
You can use filter() to apply descriptive statistics in a subset of data. For instance, you can count the number of people above 40 year old

In [28]:
df.filter(df.age < 40).count()          # just like where keyword in sql

27444

Descriptive statistics by group
Finally, you can group data by group and compute statistical operations like the mean.

In [29]:
df.groupby('age').agg({'capital-gain': 'mean'}).show()

+---+------------------+
|age| avg(capital-gain)|
+---+------------------+
| 31| 618.5954716981132|
| 85|               0.0|
| 65|3126.6021126760565|
| 53|1592.3839662447258|
| 78| 3625.294117647059|
| 34| 703.1074443591712|
| 81| 535.2162162162163|
| 28|          625.3625|
| 76| 783.5072463768116|
| 26| 298.0225498699046|
| 27|256.96347402597405|
| 44|1636.1265229615744|
| 22|253.99830220713073|
| 47|2006.0416281221092|
| 52|2367.8035230352302|
| 86|               0.0|
| 20|   66.937106918239|
| 40|1253.5745577085088|
| 57|1836.4246823956444|
| 54|1419.1957585644373|
+---+------------------+
only showing top 20 rows



Data preprocessing
Data processing is a critical step in machine learning. After you remove garbage data, you get some important insights. For instance, you know that age is not a linear function with the income. When people are young, their income is usually lower than mid-age. After retirement, a household uses their saving, meaning a decrease in income. To capture this pattern, you can add a square to the age feature

Add age square

To add a new feature, you need to:

Select the column
Apply the transformation and add it to the DataFrame

In [30]:
from pyspark.sql.functions import *
# 1 Select the column
age_square1 = df.select(col("age")**2)
# 2 Apply the transformation and add it to the DataFrame
df = df.withColumn("age_square", col("age")**2)         # withColumn is used to add in the column
df.printSchema()                                        # age_square is new field generated
df.select('age','age_square').show()

root
 |-- age: integer (nullable = true)
 |-- workclass: string (nullable = true)
 |-- fnlwgt: integer (nullable = true)
 |-- education: string (nullable = true)
 |-- educational-num: integer (nullable = true)
 |-- marital-status: string (nullable = true)
 |-- occupation: string (nullable = true)
 |-- relationship: string (nullable = true)
 |-- race: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- capital-gain: integer (nullable = true)
 |-- capital-loss: integer (nullable = true)
 |-- hours-per-week: integer (nullable = true)
 |-- native-country: string (nullable = true)
 |-- income: string (nullable = true)
 |-- age_square: double (nullable = true)



In [31]:
COLUMNS = ['age', 'age_square', 'workclass', 'fnlwgt', 'education',
           'occupation', 'relationship', 'race']
df = df.select(COLUMNS)
print(df.first())
df.show(5)

Row(age=25, age_square=625.0, workclass='Private', fnlwgt=226802, education='11th', occupation='Machine-op-inspct', relationship='Own-child', race='Black')
+---+----------+---------+------+------------+-----------------+------------+-----+
|age|age_square|workclass|fnlwgt|   education|       occupation|relationship| race|
+---+----------+---------+------+------------+-----------------+------------+-----+
| 25|     625.0|  Private|226802|        11th|Machine-op-inspct|   Own-child|Black|
| 38|    1444.0|  Private| 89814|     HS-grad|  Farming-fishing|     Husband|White|
| 28|     784.0|Local-gov|336951|  Assoc-acdm|  Protective-serv|     Husband|White|
| 44|    1936.0|  Private|160323|Some-college|Machine-op-inspct|     Husband|Black|
| 18|     324.0|        ?|103497|Some-college|                ?|   Own-child|White|
+---+----------+---------+------+------------+-----------------+------------+-----+
only showing top 5 rows



Exclude Holand-Netherlands

When a group within a feature has only one observation, it brings no information to the model. On the contrary, it can lead to an error during the cross-validation.

In [33]:
#df.select('native-country').show(5)
df.filter(col('native-country') == 'Holand-Netherlands').count()
#df.groupby('native-country').agg({'native-country': 'count'}).sort(asc("count(native-country)")).show()

1

In [ ]:
df_remove = df.filter(df.native-country !='Holand-Netherlands').show()


Build a data processing pipeline
Similar to scikit-learn, Pyspark has a pipeline API. A pipeline is very convenient to maintain the structure of the data. You push the data into the pipeline. Inside the pipeline, various operations are done, the output is used to feed the algorithm.

For instance, one universal transformation in machine learning consists of converting a string to one hot encoder, i.e., one column by a group. One hot encoder is usually a matrix full of zeroes.

The steps to transform the data are very similar to scikit-learn. You need to:

Index the string to numeric
Create the one hot encoder
Transform the data
Two APIs do the job: StringIndexer, OneHotEncoder

First of all, you select the string column to index. The inputCol is the name of the column in the dataset. outputCol is the new name given to the transformed column.

In [ ]:
StringIndexer(inputCol="workclass", outputCol="workclass_encoded")

In [ ]:
df.drop('education_num').columns

Build a data processing pipeline
Similar to scikit-learn, Pyspark has a pipeline API. A pipeline is very convenient to maintain the structure of the data. You push the data into the pipeline. Inside the pipeline, various operations are done, the output is used to feed the algorithm.

For instance, one universal transformation in machine learning consists of converting a string to one hot encoder, i.e., one column by a group. One hot encoder is usually a matrix full of zeroes.

The steps to transform the data are very similar to scikit-learn. You need to:

Index the string to numeric
Create the one hot encoder
Transform the data
Two APIs do the job: StringIndexer, OneHotEncoder

First of all, you select the string column to index. The inputCol is the name of the column in the dataset. outputCol is the new name given to the transformed column.

In [ ]:
StringIndexer(inputCol="workclass", outputCol="workclass_encoded")

In [2]:
#Fit the data and transform it
model = stringIndexer.fit(df)		
indexed = model.transform(df)

NameError: name 'stringIndexer' is not defined

In [12]:
#linear Algebra  y=mx+c             1.calculate m         2.calculate c
#linear eqaution sigmoid(1/1+e^-z)
#probability 
# Distance          root((observe-actual)^2+(O-A)^2+........+n )
#calculus 
#vector, matrix, eigen value,eigen matrix 




manage delhi weather condition for 20 years and plot the graph

SyntaxError: invalid syntax (<ipython-input-12-1854c73b0f18>, line 11)

In [14]:
import pyspark
sc.stop()
sc=pyspark.SparkContext()
txt=sc.textFile('sdj.txt')
print(txt.count())
print(txt)

6
sdj.txt MapPartitionsRDD[1] at textFile at <unknown>:0


In [ ]:
with the help of webscrapping manage data and store in dbms